In [5]:
import sys
sys.path.append("/home/sara/lib")

In [6]:
import time

import numpy
import libopf_py
import pandas as pd

from sklearn.svm import SVC
from sklearn import datasets
from sklearn.metrics import classification_report, confusion_matrix

In [7]:
#digits = datasets.load_digits()
#print("digits:   ---->>>> ", digits)

# To apply an classifier on this data, we need to flatten the image, to
# turn the data in a (samples, feature) matrix:

#n_samples = len(digits.images)
#data = digits.images.reshape((n_samples, -1))
#print("data: \n")
#print(data)
#print("n samples: \n")
#print(n_samples)
#print(len(data))


df = pd.read_csv("csv_dataset/dataframe.csv", header=None, sep=';')
df = df.drop([0], axis=0)
df = df.drop(columns=[0])
data = df.iloc[1:665,1:137]
data = data.to_numpy()
data = data.astype('float64')
target = df.iloc[1:,136]
target = target.to_numpy()
target = target.astype('float64')
n_samples = len(data)

# Debugs
print("data: \n")
print(data)
print("n samples: \n")
print(n_samples)
print(len(data))

data: 

[[217.  87. 268. ... 246. 422.   1.]
 [221.  94. 271. ... 250. 414.   1.]
 [224.  89. 274. ... 245. 433.   1.]
 ...
 [225.  88. 272. ... 236. 403.   5.]
 [220.  98. 268. ... 250. 396.   5.]
 [208.  91. 257. ... 240. 391.   5.]]
n samples: 

664
664


In [10]:
def run(split):
    n_split = int(split * n_samples)

    print("")
    print("=" * 100)
    print("")

    print("Split: %3.2f" % split)
    print("Size: %d, Classifying Size: %d, Testing Size: %d" % (n_samples, n_split, n_samples - n_split))

    rand = numpy.random.permutation(n_samples)

    random_data = data[rand]
    random_label = target[rand]

    data_train, data_test = random_data[:n_split], random_data[n_split:]
    label_train, label_test = random_label[:n_split], random_label[n_split:]

    print("-" * 20, "OPF", "-" * 20)

    def opf():
        # OPF only supports 32 bits labels at the moment
        label_train_32 = label_train.astype(numpy.int32)
        label_test_32 = label_test.astype(numpy.int32)

        O = libopf_py.OPF()

        t = time.time()
        O.fit(data_train, label_train_32)
        #    O.fit(data_train_32, label_train_32, learning="agglomerative", split=0.8)
        print("OPF: time elapsed in fitting: %f secs" % (time.time() - t))

        t = time.time()
        predicted = O.predict(data_test)
        print("OPF: time elapsed in predicting: %f secs" % (time.time() - t))

        print("Classification report for OPF:\n%s\n" % (classification_report(label_test_32, predicted)))
        print("Confusion matrix:\n%s" % confusion_matrix(label_test_32, predicted))

    opf()

    print("-" * 20, "SVM", "-" * 20)

    def _svm():
        clf = SVC()

        t = time.time()
        clf.fit(data_train, label_train)
        print("SVM: time elapsed in fitting: %f secs" % (time.time() - t))

        t = time.time()
        predicted = clf.predict(data_test)
        print("SVM: time elapsed in predicting: %f secs" % (time.time() - t))

        print("Classification report for SVM:\n%s\n" % (classification_report(label_test, predicted)))
        print("Confusion matrix:\n%s" % confusion_matrix(label_test, predicted))

    _svm()

In [11]:
run(0.8)



Split: 0.80
Size: 664, Classifying Size: 531, Testing Size: 133
-------------------- OPF --------------------
OPF: time elapsed in fitting: 0.042622 secs
OPF: time elapsed in predicting: 0.029227 secs
Classification report for OPF:
              precision    recall  f1-score   support

           1       0.48      0.36      0.41        28
           2       0.21      0.32      0.25        22
           3       0.24      0.29      0.26        28
           4       0.16      0.14      0.15        28
           5       0.33      0.26      0.29        27

   micro avg       0.27      0.27      0.27       133
   macro avg       0.28      0.27      0.27       133
weighted avg       0.29      0.27      0.27       133


Confusion matrix:
[[10  4  4  6  4]
 [ 4  7  6  3  2]
 [ 2  6  8  7  5]
 [ 4  8  9  4  3]
 [ 1  8  6  5  7]]
-------------------- SVM --------------------
SVM: time elapsed in fitting: 0.164765 secs
SVM: time elapsed in predicting: 0.016726 secs
Classification report for SVM:

/home/sara/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/home/sara/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
